In [ ]:
# Dataset: https://github.com/abachaa/MedQuAD
# Folders with empty answers: 10_MPlus_ADAM_QA, 11_MPlusDrugs_QA, 12_MPlusHerbsSupplements_QA

In [ ]:
!pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install jsonpath

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import os
import xmltodict
import re
import time
import pandas as pd
from jsonpath import jsonpath

In [ ]:
data = {
    "Questions": [],
    "Answers": [],
    "Focus": []
}

def processXmlFile(completePath):
    with open(completePath) as f:
        xmlstring = f.read()
        try:
            dataDict = xmltodict.parse(xmlstring, xml_attribs = False)
            listOfQA = json.loads(json.dumps( jsonpath(dataDict, '$..' + "QAPair")[0]))
            focus = json.loads(json.dumps( jsonpath(dataDict, '$..' + "Focus")[0]))
        except Exception as e:
            # i.e either QAPair is empty OR Focus
            return

        # if there is only single QA, then it will be dict instead of list
        if isinstance(listOfQA, dict):
            listOfQA = [listOfQA]
        
        for qaPair in listOfQA:
            try:
                # remove the extra spaces from answer with single space
                x = re.sub(' +', ' ', qaPair['Answer'])
                x = re.sub('Key Points', "", x)
                x = x.replace("\n", "").replace("-", "")
                data['Answers'].append( x )
                data['Questions'].append( qaPair['Question'] )
                data['Focus'].append(focus)
            except:
                return

In [ ]:
foldersWithEmptyAnswers = [
    "10_MPlus_ADAM_QA",
     "11_MPlusDrugs_QA",
     "12_MPlusHerbsSupplements_QA",
     "readme.txt", # As it does not contain any QAs
     "QA-TestSet-LiveQA-Med-Qrels-2479-Answers.zip", # will use it later,
     "ProcessedData.csv"
]

BASE_PATH = "/content/drive/MyDrive/MedQuad/MedQuAD-master"
for folder in os.listdir(BASE_PATH):
    if folder in foldersWithEmptyAnswers:
        continue
    else:
        print("Processing folder:", folder)
        start = time.time()

        for xmlFileName in os.listdir( os.path.join(BASE_PATH, folder) ):
            completePath = os.path.join(BASE_PATH, folder, xmlFileName)
            processXmlFile(completePath)
        
        print("Took", time.time() - start)

Processing folder: 4_MPlus_Health_Topics_QA
Took 2.751258611679077
Processing folder: 6_NINDS_QA
Took 0.7467536926269531
Processing folder: 7_SeniorHealth_QA
Took 0.2584519386291504
Processing folder: 2_GARD_QA
Took 12.15547513961792
Processing folder: 1_CancerGov_QA
Took 0.752143144607544
Processing folder: 8_NHLBI_QA_XML
Took 0.471926212310791
Processing folder: 3_GHR_QA
Took 4.997699737548828
Processing folder: 5_NIDDK_QA
Took 0.7267694473266602
Processing folder: 9_CDC_QA
Took 0.18634390830993652


In [ ]:
df = pd.DataFrame(data)
df.head()

,Questions,Answers,Focus
0,What is (are) Neck Injuries and Disorders ?,"Any part of your neck muscles, bones, joints,...",Neck Injuries and Disorders
1,What is (are) Heel Injuries and Disorders ?,Heel problems are common and can be painful. O...,Heel Injuries and Disorders
2,Do you have information about CT Scans,Summary : Computed tomography (CT) is a type o...,CT Scans
3,What is (are) Self-harm ?,Selfharm refers to a person's harming their ow...,Self-harm
4,Do you have information about Seniors' Health,Summary : People in the U.S. are living longer...,Seniors' Health


In [ ]:
df.shape

(16402, 3)

In [ ]:
df.to_csv('ProcessedData.csv', index = False)

In [ ]:
!mv ProcessedData.csv BASE_PATH